# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE
    # and then back ...
    supercell = ase_atoms(lattice, atoms, positions) * (repeat, 1, 1)
    lattice   = load_lattice(supercell)
    positions = load_positions(supercell)
    atoms = fill(Al, length(positions))

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(lattice, atoms, positions; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end

aluminium_setup (generic function with 2 methods)

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
ase_atoms(setup.model).write("al_supercell.png")

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trival statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
is_converged = DFTK.ScfConvergenceDensity(1e-4)  # Flag convergence based on density
self_consistent_field(aluminium_setup(1); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298495836147                   -0.85    4.6
  2   -8.300216368765       -2.76       -1.25    1.0
  3   -8.300447870215       -3.64       -1.87    3.2
  4   -8.300462658860       -4.83       -2.75    2.0
  5   -8.300464531342       -5.73       -3.13    3.8
  6   -8.300464591454       -7.22       -3.28    4.2
  7   -8.300464615946       -7.61       -3.42    1.1
  8   -8.300464629562       -7.87       -3.55    2.2
  9   -8.300464638812       -8.03       -3.72    1.0
 10   -8.300464641472       -8.58       -3.84    3.6
 11   -8.300464643479       -8.70       -4.09    1.8


In [4]:
self_consistent_field(aluminium_setup(2); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -16.67416648326                   -0.70    5.9
  2   -16.67862148729       -2.35       -1.14    1.5
  3   -16.67922504174       -3.22       -1.86    3.2
  4   -16.67928066065       -4.25       -2.76    3.4
  5   -16.67928606404       -5.27       -3.19    4.6
  6   -16.67928621787       -6.81       -3.49    3.5
  7   -16.67928622309       -8.28       -3.97    1.0
  8   -16.67928622431       -8.92       -4.60    1.0


In [5]:
self_consistent_field(aluminium_setup(4); is_converged, n_bands=30);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32571041739                   -0.56    6.4
  2   -33.33277879430       -2.15       -1.00    1.0
  3   -33.33553798497       -2.56       -1.68    3.8
  4   -33.33681661166       -2.89       -2.27    3.8
  5   -33.33691882148       -3.99       -2.68   13.2
  6   -33.33694386498       -4.60       -3.71    3.1
  7   -33.33694387064       -8.25       -3.96    6.2
  8   -33.33694392578       -7.26       -5.04    2.9


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298496932423                   -0.85    4.8
  2   -8.300278308482       -2.75       -1.59    1.0
  3   -8.300460471340       -3.74       -2.66    5.9
  4   -8.300439888296   +   -4.69       -2.56   11.9
  5   -8.300464393615       -4.61       -3.57    2.2
  6   -8.300464576546       -6.74       -3.90    3.4
  7   -8.300464643604       -7.17       -4.94    3.4


In [7]:
self_consistent_field(aluminium_setup(4); is_converged, mixing=SimpleMixing(), n_bands=30);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32663948887                   -0.56    6.2
  2   -33.32008283500   +   -2.18       -1.27    1.1
  3   -17.78352152797   +    1.19       -0.47    7.8
  4   -33.22548527614        1.19       -1.43    7.6
  5   -33.14468022507   +   -1.09       -1.31    3.9
  6   -27.19401228696   +    0.77       -0.67    5.2
  7   -33.33135163006        0.79       -2.05    8.0
  8   -33.33414094456       -2.55       -2.01    2.8
  9   -33.33543268590       -2.89       -2.43    2.1
 10   -33.33599605796       -3.25       -2.54    1.6
 11   -33.33658004547       -3.23       -2.75    1.9
 12   -33.33692247982       -3.47       -3.26    3.1
 13   -33.33693458135       -4.92       -3.48    3.4
 14   -33.33693853945       -5.40       -3.56    2.4
 15   -33.33693995056       -5.85       -3.70    3.2
 16   -33.33694216516       -5.65       -3.88    3.0
 17   -33.33694368801       -5.82       -4.25 

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.